# Amazon Bedrock Activation Workshop Chapter0: 基本的なモデル呼び出し

この章では Amazon Bedrock 上の基盤モデルを呼び出すための基本的な方法を体験していきます。  
なお、実行環境は SageMaker Studio Notebook の `Data Science 3.0` イメージを前提にしているため、他の環境（ローカル PC や SageMaker Notebook）で実行する場合は適宜追加モジュールのインストールや IAM の権限設定が必要になる可能性があります。

### 必要権限の付与

強めの権限にはなりますが、本コンテンツを実行するのに必要な権限を付与するために、下記ポリシーを SageMaker Studio の実行ロールにアタッチします。  

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Statement1",
            "Effect": "Allow",
            "Action": "bedrock:*",
            "Resource": "*"
        }
    ]
}
```

詳しくは[こちらのドキュメント](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)を参照してください。

## ライブラリの準備
以下のセルを実行して必要なパッケージをインストールします

In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57" \
    "langchain==0.0.320"

## AWS Python SDK (boto3)での呼び出し

Bedrock を呼び出すための boto3 client を作成します。  
Bedrock のクライアントには、Bedrock Client と Bedrock Runtime Client の2種類が存在します。 

[Bedrock Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock.html) が基盤モデル一覧の表示や、カスタマイズジョブの管理など、制御面を扱うクライアントになります。     

[Bedrock Runtime Client](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html) は基盤モデルの呼び出しに利用するクライアントになります。


In [ ]:
import boto3
import json
# bedrock client
bedrock_client = boto3.client(service_name='bedrock', region_name='us-east-1')
bedrock_runtime_client = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

In [ ]:
# 利用可能な基盤モデル一覧の表示
bedrock_client.list_foundation_models()

In [ ]:
# モデルの同期的呼び出し (Anthropic Claude-v2 の場合)

body = json.dumps({
    "prompt": "\n\nHuman:日本で一番高い山はなんですか？\n\n Assistant: ",
    # 以下のパラメータは https://docs.aws.amazon.com/ja_jp/bedrock/lus-east-1rguide/model-parameters.html#model-parameters-claude を参照
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime_client.invoke_model(
    body=body, 
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

response_body = json.loads(response.get('body').read())
print(response_body.get('completion'))

また、基盤モデルはストリーム形式で呼び出すことも可能です

In [ ]:
# モデルの逐次的呼び出し

body = json.dumps({
    "prompt": "\n\nHuman:日本で一番高い山はなんですか？\n\n Assistant: ",
    "max_tokens_to_sample": 300,
    "temperature": 0.1,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = bedrock_runtime_client.invoke_model_with_response_stream(
    body=body, 
    modelId=modelId,
    accept=accept,
    contentType=contentType
)

for body in response.get('body'):
    print(json.loads(body['chunk']['bytes'].decode())['completion'], end='')


## LangChain での呼び出し

LangChainとは、LLM の呼び出し、入出力の管理、エージェントの作成など、モデルを運用するために活用されるライブラリです。  
Bedrock も LangChain に組み込まれており、呼び出すことが可能です。

In [ ]:
from langchain.chat_models import BedrockChat
from langchain.schema import HumanMessage

In [ ]:
llm = BedrockChat(
    client=bedrock_runtime_client,
    model_id="anthropic.claude-v2",
    model_kwargs={
        "temperature":0.1,
        "max_tokens_to_sample": 1000},
)

In [ ]:
result = llm([
    HumanMessage(content="日本で一番高い山はなんですか？")
])

print(result.content)

## 様々なモデルの呼び出し

Bedrock では、テキスト生成に限らず、画像生成や埋め込み表現生成が可能です。

2023/11/29 現在、以下のモデルが呼び出し可能です。  

テキスト生成
- Anthropic Claude-v2
- Anthropic Claude Instant
- AI21 Jurassic2 Mid
- AI21 Jurassic2 Ultra
- Cohere Command
- Cohere Command Light
- Meta Llama 2 Chat 13B
- Amazon Titan Text G1 - Lite(Preview)
- Amazon Titan Text G1 - Express(Preview)

画像生成
- Stability StableDiffusion XL 0.8(Preview)
- Stability StableDiffusion XL 1.0(Preview)

埋め込み表現生成
- Amazon Titan Embeddings
- Cohere Embed English
- Cohere Embed Multilingual

### 画像生成モデルの呼び出し

In [ ]:
import base64
import io
from PIL import Image
from IPython.display import display

In [ ]:
prompt_data = "a fine image of an astronaut riding a horse on Mars"
body = json.dumps({
    "text_prompts": [{"text": prompt_data}],
    "cfg_scale": 10,
    "seed": 20,
    "steps": 50
})
modelId = "stability.stable-diffusion-xl"
accept = "application/json"
contentType = "application/json"

try:
    
    response = bedrock_runtime_client.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    
    print(response_body["result"])
    print(f'{response_body.get("artifacts")[0].get("base64")[0:80]}...')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

出力結果は base64 でエンコードされているため、デコードしてから表示します。

In [ ]:
base_64_img_str = response_body.get("artifacts")[0].get("base64")
image = Image.open(io.BytesIO(base64.decodebytes(bytes(base_64_img_str, "utf-8"))))
display(image)

### 埋め込み表現の生成

Titan Embeddings により文字列の埋め込み表現を取得することができます。  
埋め込み表現とは、文字列の性質を表現したベクトルであり、検索やレコメンデーション等で利用可能です。

In [ ]:
import numpy as np
# 入力文字列の埋め込み表現を取得する関数
def get_embedding(text: str) -> np.ndarray:
    body = json.dumps({
    "inputText": text
    })

    modelId =  "amazon.titan-embed-text-v1"
    accept = '*/*'
    contentType = 'application/json'

    response = bedrock_runtime_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
    # print(response)
    response_embeddings = np.array(response_body.get('embedding'))
    
    return response_embeddings

出力結果は1536次元のベクトルになります

In [ ]:
result = get_embedding('日本で一番高い山は？')
print(result[:20])
print('shape:', result.shape)

ベクトルの類似度を計算することで、2つの文章が意味合いとして似ている度合いを定量化することができます。  
ここでは、文書どうしの類似度計算によく使われる指標である、[コサイン類似度](https://atmarkit.itmedia.co.jp/ait/articles/2112/08/news020.html)を使って、2つの文章を比較します。  

In [ ]:
# 2つのベクトルのコサイン類似度を計算する関数
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
print(cos_sim(get_embedding('日本で一番高い山は？'), get_embedding('日本最高峰とされるのは？')))

In [ ]:
print(cos_sim(get_embedding('日本で一番高い山は？'), get_embedding('バナナはおやつに入りますか？')))

類似する文章のコサイン類似度は高く、話題が異なる文章は低くなることがわかります。  

## まとめ

このノートブックでは、AWS Python SDK(boto3) や LangChain を用いて Amazon Bedrock のモデルを呼び出すための基本的な方法をご紹介しました。  
次の章からは、より実践的に活用するための方法をご紹介していきます。